In [1]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import torchvision
import warnings
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from PIL import Image
import pathlib
import csv
# sklearn Preprocessing
from sklearn.model_selection import train_test_split
#Keras
import keras
import warnings
warnings.filterwarnings('ignore')
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
test_dataset=torchvision.datasets.ImageFolder(root='/content/drive/My Drive/img_data/test', transform=transforms.ToTensor())
train_dataset=torchvision.datasets.ImageFolder(root='/content/drive/My Drive/img_data/train', transform=transforms.ToTensor())

x, y = train_dataset[0]           
train_dataset[0][1]

0

In [3]:
print(y)

0


In [4]:
len(x[0][0])

720

In [5]:

len(test_dataset)

30

In [6]:

len(train_dataset)

1470

In [36]:
batch_size = 50
n_iters = 1000
num_epochs = 34

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)
train_dataset[303][1]   
#x,y=test_dataset[3] 
n_iters

1000

In [37]:
print(len(train_dataset[0][0][0][0]))

720


In [0]:
x,y=train_dataset[0]

In [0]:
  class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=7, stride=3, padding=3)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, stride=3, padding=3)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 8 * 20, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

In [0]:
model = CNNModel()


In [0]:
criterion = nn.CrossEntropyLoss()

In [0]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [43]:
list(model.parameters())[1].size()

torch.Size([16])

In [44]:
list(model.parameters())[2].size()

torch.Size([32, 16, 7, 7])

In [45]:
list(model.parameters())[3].size()

torch.Size([32])

In [46]:
list(model.parameters())[4].size()

torch.Size([10, 5120])

In [47]:
list(model.parameters())[5].size()

torch.Size([10])

In [48]:
iter=0
for p in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images)
            labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        iter += 1
        
        if iter % 50 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = Variable(images.cuda())
                else:
                    images = Variable(images)
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 50. Loss: 2.301560163497925. Accuracy: 16
Iteration: 100. Loss: 2.2498226165771484. Accuracy: 26
Iteration: 150. Loss: 2.1711344718933105. Accuracy: 40
Iteration: 200. Loss: 1.44114089012146. Accuracy: 56
Iteration: 250. Loss: 0.8314653635025024. Accuracy: 73
Iteration: 300. Loss: 0.5269357562065125. Accuracy: 70
Iteration: 350. Loss: 0.688021719455719. Accuracy: 76
Iteration: 400. Loss: 0.4025284945964813. Accuracy: 83
Iteration: 450. Loss: 0.44087424874305725. Accuracy: 80
Iteration: 500. Loss: 0.26891785860061646. Accuracy: 90
Iteration: 550. Loss: 0.17018213868141174. Accuracy: 93
Iteration: 600. Loss: 0.10520944744348526. Accuracy: 90
Iteration: 650. Loss: 0.12496935576200485. Accuracy: 93
Iteration: 700. Loss: 0.07496507465839386. Accuracy: 93
Iteration: 750. Loss: 0.0668041929602623. Accuracy: 90
Iteration: 800. Loss: 0.11247196048498154. Accuracy: 93
Iteration: 850. Loss: 0.09120792150497437. Accuracy: 96
Iteration: 900. Loss: 0.09041428565979004. Accuracy: 93
Iterat